In [1]:
import scanpy as scp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import scipy as sp
import pandas as pd
import umap

import sys
sys.path.append('../')
from mySCfunctions import *
from silent import *
sys.path.append('./')

import Parameters as par
from importlib import reload
reload(par);

import warnings
warnings.filterwarnings('ignore')

In [2]:
s_genes_list = \
    ['Mcm5', 'Pcna', 'Tyms', 'Fen1', 'Mcm2', 'Mcm4', 'Rrm1', 'Ung', 'Gins2',
     'Mcm6', 'Cdca7', 'Dtl', 'Prim1', 'Uhrf1', 'Mlf1ip', 'Hells', 'Rfc2',
     'Rpa2', 'Nasp', 'Rad51ap1', 'Gmnn', 'Wdr76', 'Slbp', 'Ccne2', 'Ubr7',
     'Pold3', 'Msh2', 'Atad2', 'Rad51', 'Rrm2', 'Cdc45', 'Cdc6', 'Exo1', 'Tipin',
     'Dscc1', 'Blm', 'Casp8ap2', 'Usp1', 'Clspn', 'Pola1', 'Chaf1b', 'Brip1', 'E2f8']
g2m_genes_list = \
    ['Hmgb2', 'Cdk1', 'Nusap1', 'Ube2c', 'Birc5', 'Tpx2', 'Top2a', 'Ndc80',
     'Cks2', 'Nuf2', 'Cks1b', 'Mki67', 'Tmpo', 'Cenpf', 'Tacc3', 'Fam64a',
     'Smc4', 'Ccnb2', 'Ckap2l', 'Ckap2', 'Aurkb', 'Bub1', 'Kif11', 'Anp32e',
     'Tubb4b', 'Gtse1', 'Kif20b', 'Hjurp', 'Cdca3', 'Hn1', 'Cdc20', 'Ttk',
     'Cdc25c', 'Kif2c', 'Rangap1', 'Ncapd2', 'Dlgap5', 'Cdca2', 'Cdca8',
     'Ect2', 'Kif23', 'Hmmr', 'Aurka', 'Psrc1', 'Anln', 'Lbr', 'Ckap5',
     'Cenpe', 'Ctcf', 'Nek2', 'G2e3', 'Gas2l3', 'Cbx5', 'Cenpa']

# Load the data

We upload the data with the selection of HVGs and log transformed. The log transformation of the data will make that the PCs do have a less dominant contribution among the genes.

In [4]:
a = scp.read("ManipulatedData/"+par.METRIC+"/Doublets_removed.h5ad")

# A la Marioni

In [5]:
scp.pp.log1p(a)
scp.pp.highly_variable_genes(a,flavor="seurat")
for i in s_genes_list:
    a.var.loc[a.var["Gene"]==i,"highly_variable"] = False
    
for i in g2m_genes_list:
    a.var.loc[a.var["Gene"]==i,"highly_variable"] = False
    
scp.pp.pca(a,n_comps=par.N_PCS,use_highly_variable=par.USE_HVGs)

a.obs["sample"] = a.obs["sample"].astype("str")
mnn_correct_twice(a,"sample","stage",["E8.5","E8.25","E8.0","E7.75","E7.5","E7.25","E7.0","E6.75","E6.5"])

np.save("./ManipulatedData/"+par.METRIC+"/tables/full_data_integration_Marioni_pca.npy",a.obs.loc[:,"louvain"].values.astype(str))

sample
E8.5
Performing cosine normalization...
Starting MNN correct iteration. Reference batch: 0
Step 1 of 3: processing batch 1
  Looking for MNNs...
  Computing correction vectors...
  Adjusting variance...
  Applying correction...
Step 2 of 3: processing batch 2
  Looking for MNNs...
  Computing correction vectors...
  Adjusting variance...
  Applying correction...
Step 3 of 3: processing batch 3
  Looking for MNNs...
  Computing correction vectors...
  Adjusting variance...
  Applying correction...
MNN correction complete. Gathering output...
Packing AnnData object...
Done.
E8.25
Performing cosine normalization...
Starting MNN correct iteration. Reference batch: 0
Step 1 of 2: processing batch 1
  Looking for MNNs...
  Computing correction vectors...
  Adjusting variance...
  Applying correction...
Step 2 of 2: processing batch 2
  Looking for MNNs...
  Computing correction vectors...
  Adjusting variance...
  Applying correction...
MNN correction complete. Gathering output...
Pac

ValueError: Value passed for key 'X_MNN' is of incorrect shape. Values of obsm must match dimensions (0,) of parent. Value had shape (111622, 50) while it should have had (119836,).

In [ ]:
a.obsm["X_pca_MNN"] = np.load("./ManipulatedData/"+par.METRIC+"/tables/full_data_integration_Marioni_pca.npy")

scp.pp.neighbors(a,use_rep="X_MNN",n_pcs=par.N_PCS,knn=par.N_NEIGBOURS)

scp.tl.umap(a)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=[20,20])

sb.scatterplot(a.obsm["X_umap"][:,0],a.obsm["X_umap"][:,1],a.obs.loc[:,"stage"])

# A la Klein

In [4]:
from sklearn import preprocessing
from sklearn.decomposition import PCA, TruncatedSVD

In [5]:
l = (a.obs["stage"]=="E8.5") + (a.obs["stage"]=="E8.25") 
b = a[l,:].copy()

In [6]:
scp.pp.log1p(b)
scp.pp.highly_variable_genes(b,flavor=par.HVG_METHOD)

In [7]:
X = b[b.obs["stage"]=="E8.5",b.var["highly_variable"]].X.copy()
X2 = b[b.obs["stage"]=="E8.25",b.var["highly_variable"]].X.copy()

In [8]:
X_norm = preprocessing.MaxAbsScaler().fit_transform(X)
X2_norm = preprocessing.MaxAbsScaler().fit_transform(X2)

In [9]:
model = TruncatedSVD(n_components=par.N_PCS).fit(X_norm)

In [10]:
X_pca = model.fit_transform(X_norm)
X2_pca = model.fit_transform(X2_norm)

In [11]:
b.obsm["X_pca"] = np.zeros([b.shape[0],par.N_PCS])
b.obsm["X_pca"][b.obs["stage"]=="E8.5",:] = X_pca
b.obsm["X_pca"][b.obs["stage"]=="E8.25",:] = X2_pca

In [12]:
#Make nearest neighbours (1)
scp.pp.neighbors(b,metric=par.METRIC,knn=200,use_rep="X_pca",n_pcs=par.N_PCS)
d = b.obsp["distances"].copy()
d

<32588x32588 sparse matrix of type '<class 'numpy.float64'>'
	with 456232 stored elements in Compressed Sparse Row format>

In [13]:
#Cutoff local (2)
maxDistance = 3*(d.power(-1).max(axis=1).power(-1)).toarray()
loc_remove = sp.sparse.csc_matrix((d > maxDistance)).nonzero()
d[loc_remove] = 0
d.eliminate_zeros()
d

<32588x32588 sparse matrix of type '<class 'numpy.float64'>'
	with 455744 stored elements in Compressed Sparse Row format>

In [14]:
#Cutoff global (3)
s1 = b.obs["stage"]=="E8.5"
s2 = b.obs["stage"]=="E8.25"

#Remove between samples
mean_across_samples = (d[s1][:,s2].sum()+d[s2][:,s1].sum())/(d[s1][:,s2].data.shape[0]+d[s2][:,s1].data.shape[0])

loc = (np.array([i for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s1[i] and s2[j]])
       ,np.array([j for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s1[i] and s2[j]]))

v = d[loc]
v[d[loc] > mean_across_samples] = 0
d[loc] = v

loc = (np.array([i for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s2[i] and s1[j]])
       ,np.array([j for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s2[i] and s1[j]]))

v = d[loc]
v[d[loc] > mean_across_samples] = 0
d[loc] = v

#Remove within sample
meanDistance = d[s1][:,s1].sum()/d[s1][:,s1].data.shape[0]
stdDistance = np.sqrt(d[s1][:,s1].power(2).sum()/d[s1][:,s1].data.shape[0]-np.power(d[s1][:,s1].sum()/d[s1][:,s1].data.shape[0],2))
maxDistance_s1 = (meanDistance+stdDistance)

meanDistance = d[s2][:,s2].sum()/d[s2][:,s2].data.shape[0]
stdDistance = np.sqrt(d[s2][:,s2].power(2).sum()/d[s2][:,s2].data.shape[0]-np.power(d[s2][:,s2].sum()/d[s2][:,s2].data.shape[0],2))
maxDistance_s2 = (meanDistance+stdDistance)

loc = (np.array([i for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s1[i] and s1[j]])
       ,np.array([j for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s1[i] and s1[j]]))

for i,j in zip(loc[0],loc[1]):
    if d[i,j] > maxDistance_s1:
        d[i,j] = 0
        
loc = (np.array([i for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s2[i] and s2[j]])
       ,np.array([j for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s2[i] and s2[j]]))

for i,j in zip(loc[0],loc[1]):
    if d[i,j] > maxDistance_s2:
        d[i,j] = 0
        
d.eliminate_zeros()
d

<32588x32588 sparse matrix of type '<class 'numpy.float64'>'
	with 395284 stored elements in Compressed Sparse Row format>

In [18]:
#Limit to 20 MNN
s1,s2 = d.nonzero()
d[s1,s2] = d[s2,s1]
d.eliminate_zeros()
d

<32588x32588 sparse matrix of type '<class 'numpy.float64'>'
	with 212660 stored elements in Compressed Sparse Row format>

# A la Klein modified

In [4]:
from sklearn import preprocessing
from sklearn.decomposition import PCA, TruncatedSVD

In [5]:
l2 = ['E8.5', 'E8.25', 'E8.0', 'E7.75', 'E7.5', 'E7.25', 'E7.0', 'E6.75']
l1 = ['E8.25', 'E8.0', 'E7.75', 'E7.5', 'E7.25', 'E7.0', 'E6.75', 'E6.5']

In [9]:
data_edges = pd.DataFrame()

for st1,st2 in zip(l1,l2):

    #Extract datasets
    l = (a.obs["stage"]==st1) + (a.obs["stage"]==st2) 
    b = a[l,:].copy()
    
    #Make list of cells for each stage
    s1 = b.obs["stage"]==st2
    s2 = b.obs["stage"]==st1
    
    #Log normalize
    scp.pp.log1p(b)
    
    #Extract HVGs of time i+1
    b2 = b[b.obs["stage"]==st2,:].copy()
    scp.pp.highly_variable_genes(b2,flavor=par.HVG_METHOD)
    hvg = b2.var["highly_variable"].values
    del b2
    
    #Make PCs of time i+1 and project the space of i
    X = b[b.obs["stage"]==st1,hvg].X.copy()
    X2 = b[b.obs["stage"]==st2,hvg].X.copy()

    X_norm = preprocessing.MaxAbsScaler().fit_transform(X)
    X2_norm = preprocessing.MaxAbsScaler().fit_transform(X2)
    
    model = TruncatedSVD(n_components=par.N_PCS).fit(X2_norm) #Fit with the later time point
    
    X_pca = model.transform(X_norm)
    X2_pca = model.transform(X2_norm)
    
    b.obsm["X_pca"] = np.zeros([b.shape[0],par.N_PCS])
    b.obsm["X_pca"][b.obs["stage"]==st1,:] = X_pca
    b.obsm["X_pca"][b.obs["stage"]==st2,:] = X2_pca
    
    #Make neighbours between stages (1)
    scp.external.pp.bbknn(b,"stage",metric=par.METRIC,neighbors_within_batch=10,use_rep="X_pca",n_pcs=par.N_PCS)
    d = b.obsp["distances"].copy()
    
    #Cutoff local (2)
    maxDistance = 3*(d.power(-1).max(axis=1).power(-1)).toarray()
    loc_remove = sp.sparse.csc_matrix((d > maxDistance)).nonzero()
    d[loc_remove] = 0
    d.eliminate_zeros()
    
    #Cutoff global (3)
        #Remove between samples
    mean_across_samples = (d[s1][:,s2].sum()+d[s2][:,s1].sum())/(d[s1][:,s2].data.shape[0]+d[s2][:,s1].data.shape[0])

    loc = (np.array([i for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s1[i] and s2[j]])
           ,np.array([j for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s1[i] and s2[j]]))

    v = d[loc]
    v[d[loc] > mean_across_samples] = 0
    d[loc] = v

    loc = (np.array([i for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s2[i] and s1[j]])
           ,np.array([j for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s2[i] and s1[j]]))

    v = d[loc]
    v[d[loc] > mean_across_samples] = 0
    d[loc] = v

        #Remove within sample
    meanDistance = d[s1][:,s1].sum()/d[s1][:,s1].data.shape[0]
    stdDistance = np.sqrt(d[s1][:,s1].power(2).sum()/d[s1][:,s1].data.shape[0]-np.power(d[s1][:,s1].sum()/d[s1][:,s1].data.shape[0],2))
    maxDistance_s1 = (meanDistance+stdDistance)

    meanDistance = d[s2][:,s2].sum()/d[s2][:,s2].data.shape[0]
    stdDistance = np.sqrt(d[s2][:,s2].power(2).sum()/d[s2][:,s2].data.shape[0]-np.power(d[s2][:,s2].sum()/d[s2][:,s2].data.shape[0],2))
    maxDistance_s2 = (meanDistance+stdDistance)

    loc = (np.array([i for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s1[i] and s1[j]])
           ,np.array([j for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s1[i] and s1[j]]))

    for i,j in zip(loc[0],loc[1]):
        if d[i,j] > maxDistance_s1:
            d[i,j] = 0

    loc = (np.array([i for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s2[i] and s2[j]])
           ,np.array([j for i,j in zip(d.nonzero()[0],d.nonzero()[1]) if s2[i] and s2[j]]))

    for i,j in zip(loc[0],loc[1]):
        if d[i,j] > maxDistance_s2:
            d[i,j] = 0

    d.eliminate_zeros()
    
    #Remove excess of neighbours 
    #s1,s2 = d.nonzero()
    #d[s1,s2] = d[s2,s1]
    #d.eliminate_zeros()    
    
    data_edges_aux = pd.DataFrame()
    data_edges_aux["source"] = b.obs.index[d.nonzero()[0]]
    data_edges_aux["target"] = b.obs.index[d.nonzero()[1]]
    data_edges_aux["value"] = d.data
    
    data_edges = data_edges.append(data_edges_aux)

In [10]:
data_edges.to_csv("./ManipulatedData/"+par.METRIC+"/tables/Klein_network_edges.csv")